In [30]:
import whisper
import torch

from utils import log_mel_spectrogram, load_audio, pad_or_trim
from whisper.decoding import DecodingOptions
from model import Whisper
from whisper.tokenizer import get_tokenizer


model_original = whisper.load_model("tiny")
model_original.to(torch.float32)

tokenizer = get_tokenizer(model_original.is_multilingual, language='en', task='transcribe')

model = Whisper.load_from_pretrained("tiny") 

In [16]:
mel = log_mel_spectrogram(load_audio('./audio.mp3'))
mel_segment = pad_or_trim(mel, 30 * 16000 // 160).unsqueeze(0)
model_original.decode(mel_segment, DecodingOptions(fp16=False))[0].text

In [41]:
original_encoder_logits = model_original.encoder(mel_segment)
encoder_logits = model.encoder(mel_segment)

assert torch.allclose(original_encoder_logits, encoder_logits)

tokens = torch.randint(0, 50000, (1,15))
original_decoder_logits = model_original.decoder(tokens, original_encoder_logits)
decoder_logits = model.decoder(tokens, encoder_logits)

assert torch.allclose(original_decoder_logits, decoder_logits)

encoder_logits = model.encoder(mel_segment)
tokenizer.decode(model.generate(torch.tensor([tokenizer.sot_sequence], dtype=torch.int64), encoder_logits)[0,4:]).strip()

'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the exhibition.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'